In [ ]:
import json
import os
import google.generativeai as genai


WORDS_RAW_PATH = "/words"
SENTENCE_RAW_PATH = "/sentence"

ENDPOINT_NAME = os.getenv("ENDPOINT")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

def lambda_handler(event, context):
    genai.configure(api_key=GOOGLE_API_KEY)
    model = genai.GenerativeModel('gemini-1.5-pro-latest')
    inputs = json.loads(event["body"])["inputs"] 
    input_id_images = json.loads(event["body"])["input_id_images"] 
    
    if event["rawPath"] == WORDS_RAW_PATH:
        chat = model.start_chat(history=words_prompt_history())
        inputs = chat.send_message(inputs)
    elif event["rawPath"] == SENTENCE_RAW_PATH:
        chat = model.start_chat(history=sentence_prompt_history())
        inputs = chat.send_message(inputs)
    
    translate_input = change_prompt(inputs, input_id_images)
    response = runtime.invoke_endpoint(EndpointName=ENDPOINT_NAME, 
                                        ContentType='application/json', 
                                        Accept='application/json',
                                        Body=json.dumps(translate_input))
    result = []
    statusCode = response['ResponseMetadata']['HTTPStatusCode']
    if statusCode == 200:
        response_payload = response['Body'].read().decode('utf-8')
        result.append(json.loads(response_payload)["images"])
    return {
        'statusCode': statusCode,
        'body': json.dumps(result)
    }
    
def words_prompt_history():
    return [
        {
            "role": "user",
            "parts": ["You're a fashion expert vividly describing outfits for given styles in English. \n\nExamples: 캐주얼 패션"]
        },
        {
            "role": "model",
            "parts": ["A relaxed yet trendy look: Oversized graphic tee in faded colors, ripped mom jeans, chunky sneakers, crossbody bag, layered necklaces, messy beach waves hairstyle."]
        },
        {
            "role": "user",
            "parts": ["스트릿 패션"]
        },
        {
            "role": "model",
            "parts": ["An edgy urban look: Oversized hoodie with bold graphics, distressed skinny jeans, high-top sneakers, baseball cap, statement accessories like chunky chains and rings, windswept hairstyle."]
        }
    ]
def sentence_prompt_history():
    return [
        {
            "role": "user",
            "parts": ["You're a fashion expert vividly describing outfits for given styles, translating this sentence from Korean to English as well. \n\n Examples: 캐주얼 패션으로 스타일링, 느긋하지만 트렌디한 룩: 빛이 바랜 색상의 오버사이즈 그래픽 티셔츠, 찢어진 모미 진"]
        },
        {
            "role": "model",
            "parts": ["A relaxed yet trendy look: Oversized graphic tee in faded colors, ripped mom jeans, chunky sneakers, crossbody bag, layered necklaces, messy beach waves hairstyle."]
        },
        {
            "role": "user",
            "parts": ["스트릿 패션의 스타일링, 도시적이고 쿨한 룩: 대담한 그래픽이 들어간 오버사이즈 후디, 낡은 스키니"]
        },
        {
            "role": "model",
            "parts": ["An edgy urban look: Oversized hoodie with bold graphics, distressed skinny jeans, high-top sneakers, baseball cap, statement accessories like chunky chains and rings, windswept hairstyle."]
        }
    ]
def change_prompt(inputs, images):
    return {
        "inputs": inputs,
        "input_id_images": [img for img in images]
    }